<a href="https://colab.research.google.com/github/Ditsuhi/ConvLSTM_Madrid/blob/main/ConvLSTMadrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rarfile
import rarfile

with rarfile.RarFile('/content/drive/MyDrive/csvFiles.rar', 'r') as rar_ref:
    rar_ref.extractall('/content/')

In [ ]:
from glob import glob
import numpy as np
import pandas as pd
import re


airMet = glob("/content/csvFiles/*.csv")


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers
from keras.regularizers import l2
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import ConvLSTM2D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from keras.callbacks import ModelCheckpoint
import tensorflow as tf 
from numpy.random import seed
from operator import itemgetter


def modelarch():
  model = keras.Sequential([keras.Input(shape=(None,  340, 2, 1)),      
  layers.ConvLSTM2D(filters=16, kernel_size=(3, 3), padding="same", return_sequences=True, kernel_regularizer=l2(0.01)),
  layers.Dropout(0.5),
  layers.BatchNormalization(),
  layers.ConvLSTM2D(filters=32, kernel_size=(3, 3), padding="same", return_sequences=True),
  layers.BatchNormalization(),
  layers.ConvLSTM2D(filters=32, kernel_size=(3, 3), padding="same", return_sequences=True),
  layers.BatchNormalization(),
  layers.ConvLSTM2D(filters=16, kernel_size=(3, 3), padding="same", return_sequences=True),
  layers.BatchNormalization(),      
  layers.ConvLSTM2D(filters=1, kernel_size=(1, 1), activation='relu')])
  model.compile(optimizer="adam", loss='mse')
  return model

  
def sortingFiles(eachFile):
    return int(eachFile) if eachFile.isdigit() else eachFile


def natural_keys(eachFile):
    return [ sortingFiles(c) for c in re.split('(\d+)',eachFile)]

def sortedData(data):
  sorted_airMetTraf = sorted(data, key = natural_keys)
  return sorted_airMetTraf 

def pieceData (dataComplete):
  sorted_part = dataComplete[: 4344]
  return sorted_part

def concatAllData(sort_data): 
  indices_to_access = [1, 5]
  # df = [pd.read_csv(f, usecols=itemgetter(*b)(a)).values for f in sortedData(sort_data)]
  df = [pd.read_csv(f, usecols=range(1, 3)).values for f in sortedData(sort_data)]
  df_all = np.asarray(df)
  print(df_all.shape)
  return df_all


def split_sequence(data):
	X, y = list(), list()
	for i in range(len(data)):    
		end_ix = i + 24    
		if end_ix+24 > len(data)-1:
			break
		seq_x, seq_y = data[i:end_ix], data[end_ix+24]
		X.append(seq_x)
		y.append(seq_y)
    
	return np.array(X), np.array(y)
 

def splitTrainTest(X, y):
  X_train_notNorm, X_test_notNorm, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle = False)
  return X_train_notNorm, X_test_notNorm, y_train, y_test


def normMinMax (X_train_notNorm, X_test_notNorm):
  scaler = MinMaxScaler(feature_range=(0, 1), copy = False)
  train_Normalised = X_train_notNorm.reshape(-1, 340*2)
  test_Normalised = X_test_notNorm.reshape(-1, 340*2)
  train_scaled = scaler.fit_transform(train_Normalised)
  test_scaled = scaler.transform(test_Normalised)
  X_train = train_scaled.reshape(X_train_notNorm.shape[0], X_train_notNorm.shape[1], X_train_notNorm.shape[2], X_train_notNorm.shape[3])
  X_test = test_scaled.reshape(X_test_notNorm.shape[0], X_test_notNorm.shape[1], X_test_notNorm.shape[2], X_test_notNorm.shape[3])
  return X_train, X_test


def reshapeData (X_train, y_train):
  X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 340, 2, 1))
  y_train_reshaped = y_train.reshape((y_train.shape[0], 340, 2, 1))
  X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 340, 2, 1))
  y_test_reshaped = y_test.reshape(y_test.shape[0], 340*2)
  return X_train_reshaped, y_train_reshaped, X_test_reshaped, y_test_reshaped 


def earlystop():
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, restore_best_weights=True)
  return es

  
# total_testscore =[]
# seed_value = 42
# tf.random.set_seed(seed_value)
# seed(seed_value)
concatData =concatAllData(pieceData(airMet))
X, y = split_sequence(concatData)
X_train_notNorm, X_test_notNorm, y_train, y_test = splitTrainTest(X, y)
X_train, X_test = normMinMax (X_train_notNorm, X_test_notNorm)
X_train_reshaped, y_train_reshaped, X_test_reshaped, y_test_reshaped = reshapeData (X_train, y_train)
checkpoint = ModelCheckpoint("{epoch:02d}-{val_loss:.2f}.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', save_freq = "epoch")
modelarch().fit(X_train_reshaped, y_train_reshaped,  epochs=100, verbose=2, validation_split=0.1, shuffle=False, callbacks=[earlystop(), checkpoint])
# modelarch().load_weights("/content/02-2424.39.hdf5")
yhat = modelarch(seed_value).predict(X_test_reshaped , verbose=1)
yhats = yhat.reshape(y_test.shape[0], 340*2)
testScore = mean_squared_error(yhats, y_test_reshaped, squared=False)
total_testscore.append(testScore)
